# TF-IDF 구하기
- 벡터화하기 전에 데이터 전처리
   - document별 형태소 분석
   - 미등록단어 처리
   - 명사만 추출
   - 불용어(stopwords) 처리
- document별 단어로 tokenize되어 있는 것을 다시 합쳐서 각각 하나의 document로 만들기
- 벡터화(vectorize) 하기
- tf-idf 값 구하기

In [63]:
import pandas as pd

In [211]:
# CSV 데이터 불러오기
clova_df = pd.read_csv("../../data/sentiment/sentiment_clova_new.csv", index_col=0)
kakao_df = pd.read_csv("../../data/sentiment/sentiment_kakao_new.csv", index_col=0)
geni_df = pd.read_csv("../../data/sentiment/sentiment_gigagenie_new.csv", index_col=0)
nugu_df = pd.read_csv("../../data/sentiment/sentiment_nugu_new.csv", index_col=0)

In [231]:
nugu_df.url[0]

'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=277&aid=0003818204'

In [261]:
temp = nugu_df

In [264]:
t = temp[temp.url.notnull()].url.str.contains('ppang7942')
del_index = temp[temp.url.notnull()][t].index

In [266]:
print(len(nugu_df))
print(len(nugu_df.drop(del_index)))

10174
10131


In [267]:
nugu_df.drop(del_index, inplace=True)

In [268]:
# 데이터 합치기
# 긍정 document
pos_c_text = clova_df.positive_text[clova_df.positive_text.notnull()]
pos_k_text = kakao_df.positive_text[kakao_df.positive_text.notnull()]
pos_g_text = geni_df.positive_text[geni_df.positive_text.notnull()]
pos_n_text = nugu_df.positive_text[nugu_df.positive_text.notnull()]

positive_text = [pos_c_text, pos_k_text, pos_g_text, pos_n_text]

In [269]:
# 데이터 합치기
# 부정 document
neg_c_text = clova_df.negative_text[clova_df.negative_text.notnull()]
neg_k_text = kakao_df.negative_text[kakao_df.negative_text.notnull()]
neg_g_text = geni_df.negative_text[geni_df.negative_text.notnull()]
neg_n_text = nugu_df.negative_text[nugu_df.negative_text.notnull()]

negative_text = [neg_c_text, neg_k_text, neg_g_text, neg_n_text]

In [270]:
len(positive_text), len(negative_text)

(4, 4)

# <br>
## 긍정 document

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [272]:
komoran.pos("네콘")

[('네콘', 'NNG')]

In [273]:
komoran.pos("척수 장애인") 

[('척수', 'NNP'), ('장애인', 'NNP')]

In [274]:
komoran.pos("가스잠그미") 

[('가스잠그미', 'NNG')]

In [275]:
komoran.pos("삼행시") 

[('삼행시', 'NNG')]

In [276]:
komoran.pos("교동제비집")

[('교동제비집', 'NNG')]

In [277]:
komoran.pos("교동스튜디오")

[('교동', 'NNP'), ('스튜디오', 'NNP')]

In [278]:
komoran.pos("올레tv스카이라이프")

[('올레tv스카이라이프', 'NNG')]

In [279]:
from konlpy.tag import Komoran

In [311]:
komoran = Komoran(userdic='./unregistered.txt')

In [312]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

results = []
for i, brand_text in enumerate(positive_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)
    
    docs = []
    for doc in pos_tagging_list:
        want_words = []
        for word, pos in doc:
            if p.match(pos):
                want_words.append(word)
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

In [313]:
results = []
for i, brand_text in enumerate(positive_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

pos tagging - 0
noun extract - 0
finish - 0
pos tagging - 1
noun extract - 1
finish - 1
pos tagging - 2
noun extract - 2
finish - 2
pos tagging - 3
noun extract - 3
finish - 3


### pickle로 일단 저장

In [314]:
import pickle
pickle.dump(results, open('tfidf_pos_list4.pickle', "wb"))

In [315]:
load_result = pickle.load(open('tfidf_pos_list4.pickle', "rb"))
results = load_result

<br/><br/>
### stopword 제거

In [285]:
'qoo' in stopwords

True

In [316]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

final_docs = []
for doc in results:
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in unique_NN_words:
        if word in stopwords:
            while word in final_NN_words: final_NN_words.remove(word)
    final_docs.append(final_NN_words)

from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            while True:
                try:
                    final_NN_words.remove(word)
                except ValueError:
                    break
    print("finish -", i)

In [317]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    print("finish -", i)

Widget Javascript not detected.  It may not be installed or enabled properly.


finish - 0


Widget Javascript not detected.  It may not be installed or enabled properly.


finish - 1


Widget Javascript not detected.  It may not be installed or enabled properly.


finish - 2


Widget Javascript not detected.  It may not be installed or enabled properly.


finish - 3


<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [318]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [319]:
documents[0][:200]

'직장인 김 모 미국 출장 당시 인공지능 AI 스피커 구글 홈 사 아내 아들 영어 공부 부탁 김 영어 유치원 아들 구글 홈 대화 학부모 조언 스피커 정보 확인 검색 음악 알람 아이 흥미 영어 사용 대학 강모 네이버 스피커 웨이브 이벤트 선물 내년 교환 학생 계획 강 웨이브 영어 회화 기능 활용 거란 강 네이버 번역 파파 활용 기본 회화 습득 활용 웨이브 대화'

<br/><br/>
### 벡터화 하기

In [320]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [321]:
cv=CountVectorizer(max_df=0.85, max_features=10000, stop_words=stopwords)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

/Users/Taewoo/anaconda2/envs/python36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_flash_removecallback', 'gyste7gb', 'kbs', 'mbc뉴스', 'news1', 'sbs', 'sbs화면캡처무기수', '무단'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [322]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

10000

In [323]:
cv.vocabulary_

{'김진형': 3866,
 'airi': 64,
 '코리': 8768,
 '하현': 9404,
 '팬서': 9122,
 '스트레인지': 6313,
 'uiot': 2611,
 '원혁': 7238,
 'aiot': 60,
 '수요처': 6198,
 '최준': 8604,
 'luxmen': 1479,
 'personalizedtext': 1860,
 '임미숙': 7694,
 'ups': 2649,
 'nlskrl': 1695,
 '형석': 9733,
 '투자비': 8999,
 '패트릭': 9118,
 '피력': 9309,
 '십수': 6496,
 '한나절': 9481,
 '상층부': 5841,
 '불평등': 5552,
 '플리커': 9300,
 'items': 1229,
 '중략': 8244,
 'inside': 1186,
 '하버드': 9367,
 '오즈': 7042,
 '이한': 7587,
 'mashine': 1511,
 '뒷심': 4475,
 'wake': 2778,
 '명사': 4863,
 '뇌리': 4098,
 '간밤': 3059,
 '박흥순': 5163,
 '김경민': 3776,
 '여세': 6867,
 '국제가전박람회': 3604,
 '문지현': 5001,
 '김기진': 3782,
 '동승': 4412,
 '김승미': 3815,
 '쉬리': 6258,
 '선희': 5970,
 'weconomy': 2807,
 '마이크로소프트웨어': 4732,
 '셰어': 6065,
 '에디터톤': 6811,
 'marathon': 1501,
 '분장': 5528,
 '태평로': 8925,
 'makers': 1488,
 '한정수': 9531,
 '라지브': 4551,
 '콤비네이션': 8795,
 '특파원': 9038,
 '신현규': 6452,
 '희영': 9984,
 '특전': 9035,
 '홍익대': 9818,
 'walt': 2785,
 '진원지': 8365,
 '편재': 9162,
 '함의': 9565,
 '김수연': 3812,
 '도요타자동차': 4353,

<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [324]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [325]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

uiot                 0.6073192313452243
아들과딸                 0.26650524890303295
네콘                   0.25811253581170945
가스잠그미                0.11035246096297725
브라보라이프               0.10438299288746042
척수장애인                0.08042636985437325
tomatalk             0.07354256317071074
원빈                   0.06965478096329271
cloi                 0.06359784348822377
첼로                   0.05602667164438761
아들과                  0.05299820290685314
전기장판                 0.04996973416931868
브라우니                 0.04845549980055145
매스노트                 0.047446814948845645
cic                  0.046941265431784215
목돈                   0.046941265431784215
이원준                  0.04675951735719375
중학                   0.04542703106301699
진에어                  0.04542703106301699
aiaas                0.044889136662906

오라떼                  0.1497641790559481
임채무                  0.12302057565310023
높임말                  0.11767185497253066
동래                   0.0990065546331874
삼행시              

- 추가 
   - 클로바
     1. 네콘
     2. 척수장애인
     3. 척수 장애인
     4. 가스잠그미
     5. 브라보라이프
   - 카카오
     1. 산타토익
     2. 산타 토익
     3. qm3 (뭔가 관련있는듯)
     4. qm6 (뭔가 관련있는듯)
     5. 삼행시
   - 기가지니
     1. 바바북
     2. 바바펜
     3. 바바키트
     4. 교동 기가 아일랜드
     5. 올레tv스카이라이프
   - 누구
     1. 아트리체 (아파트 관련) \- '청계' 라는 단어와도 연관(청계 다우 아트리체)
     2. qoo(qoo10 관련 사이트 때문에)
     3. 범양건영 (아파트 관련)
     
- 삭제
   - 클로바
       1. li
       2. txt
       3. 유조
       4. 엠게임
       5. date
   - 카카오
       6. 조수용 (카카오 공동 대표)
       7. 배틀필드 (관련성이 전혀 없는 것 같은데 자꾸 나옴)
       8. 이석영 (카카오 AI 서비스 팀장)
       9. rx (관련없어보임... 근데 많이 나오긴 함)
       10. 콰르텟 (전혀 관련없는 "카카오프렌즈 콰르텟 강남점에서 라이언에그번을 맛보다" 이런거 많이 나옴)
       
- 렘런트유노마켓 blog.naver.com/ppang7942/ 블로그 데이터 삭제하고싶다.

# <br>
## 부정 document

<br/><br/>
### POS tagging하기

#### 1.  Komoran 사용

In [131]:
komoran.pos("척수 장애인")

[('척수', 'NNP'), ('장애인', 'NNP')]

In [132]:
komoran.pos("척수장애인")

[('척수', 'NNP'), ('장애인', 'NNP')]

In [86]:
komoran.pos("무드등")

[('무드', 'NNG'), ('등', 'NNB')]

In [123]:
komoran.pos("반려견") 

[('반려', 'NNG'), ('견', 'NNG')]

In [124]:
komoran.pos("샤오미")

[('샤오미', 'NNP')]

In [51]:
komoran.pos("척수 장애인") 

[('척수', 'NNP'), ('장애인', 'NNP')]

In [105]:
komoran.pos("nugu")

[('nugu', 'SL')]

In [122]:
komoran.pos("line out") 

[('line out', 'NNG')]

In [130]:
komoran.pos("null")

[('null', 'SL')]

In [126]:
komoran.pos("스포티지")

[('스포', 'NNP'), ('티', 'NNG'), ('지', 'NNB')]

In [195]:
komoran.pos("메를로랩")

[('메를로랩', 'NNG')]

In [61]:
komoran.pos("올레tv스카이라이프")

[('올레', 'NNP'), ('tv', 'SL'), ('스카이', 'NNP'), ('라이프', 'NNP')]

In [87]:
from konlpy.tag import Komoran

In [327]:
komoran = Komoran(userdic='./unregistered.txt')

In [328]:
import re

# 명사와 외래어만 뽑아내기위한 정규표현식
p = re.compile('NN.*|SL')

results = []
for i, brand_text in enumerate(positive_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)
    
    docs = []
    for doc in pos_tagging_list:
        want_words = []
        for word, pos in doc:
            if p.match(pos):
                want_words.append(word)
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

In [329]:
results = []
for i, brand_text in enumerate(negative_text):
    pos_tagging_list = list(map(komoran.pos, brand_text))
    print("pos tagging -", i)

    docs = []
    for doc in pos_tagging_list:
        want_words = [word for word, pos in doc if p.match(pos)]         
        docs.append(want_words)
    print("noun extract -", i)
    
    results.append(sum(docs, []))
    print("finish -", i)

pos tagging - 0
noun extract - 0
finish - 0
pos tagging - 1
noun extract - 1
finish - 1
pos tagging - 2
noun extract - 2
finish - 2
pos tagging - 3
noun extract - 3
finish - 3


### pickle로 일단 저장

In [330]:
import pickle
pickle.dump(results, open('tfidf_neg_list3.pickle', "wb"))

In [331]:
load_result2 = pickle.load(open('tfidf_neg_list3.pickle', "rb"))
results = load_result2

<br/><br/>
### stopword 제거

In [332]:
stopwords = [line.strip() for line in open('./stopwordsKor.txt', encoding='utf-8')]

final_docs = []
for doc in results:
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in unique_NN_words:
        if word in stopwords:
            while word in final_NN_words: final_NN_words.remove(word)
    final_docs.append(final_NN_words)

from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            while True:
                try:
                    final_NN_words.remove(word)
                except ValueError:
                    break
    print("finish -", i)

In [333]:
from tqdm import tqdm_notebook
stopword_dict = {}
for s_word in stopwords:
    stopword_dict[s_word] = 1

final_docs = []
for i, doc in enumerate(results):
    unique_NN_words = set(doc)
    final_NN_words = doc
    
    for word in tqdm_notebook(unique_NN_words):
        if stopword_dict.get(word):
            final_NN_words = list(filter(lambda x: x!= word, final_NN_words))
    final_docs.append(final_NN_words)
    #print("finish -", i)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


<br/><br/>
### document별 단어들을 각각의 document로 바꾸기

In [334]:
documents = []
for doc_words in final_docs:
    document = " ".join(doc_words)
    documents.append(document)

In [335]:
documents[0][:200]

'유선 래 연결 와이파이 연결 사용 음성 인식 스피커 일상 차지 역할 흐름 가운데 국내 업체 세계 시장 맥 지적 전문가 국내 업체 시장 확대 이유 길 지적 글 사기 기본 사례 네이버 카카오 날 하루 수준 카메라 인식 로그인 질문 마찬가지 대답 네이버 음악 듣기 결제 상태 지원 별도 이퀄라이저 설정 지하철역 자동차 추천 경로 검색 비교 효과 결과 근처 분식집 '

<br/><br/>
### 벡터화 하기

In [336]:
from sklearn.feature_extraction.text import CountVectorizer     # 벡터 수 카운트 할 때

In [337]:
cv=CountVectorizer(max_df=0.85, max_features=10000, stop_words=stopwords)     
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW(Bag Of Words) 인코딩한 벡터를 만든다.
# max_df=0.85란 말은 documents들에서 85%이상 나타나는 토큰(단어)를 무시하라는 것, 
# max_df=25란 말은 documents들에서 25번 이상 나타나는 토큰(단어)를 무시하라는 것,
# maximum 단어의 개수를 1만개로 정함 - 빈도수가 높은 단어 순으로 1만개를 자름(메모리 에러 안나려고)
# min_df=1, ngram_range=(1,1) 등은 default 값임  

word_count_vector=cv.fit_transform(documents)      # vectorize된 word_count_vector
feature_names=cv.get_feature_names()

/Users/Taewoo/anaconda2/envs/python36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_flash_removecallback', 'gyste7gb', 'kbs', 'mbc뉴스', 'news1', 'sbs', 'sbs화면캡처무기수', '무단'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [338]:
len(cv.vocabulary_)      # 위에서 max_features를 10000으로 주었지만 단어 수가 그 이하였음

9781

In [339]:
cv.vocabulary_

{'듣기': 2906,
 '이퀄라이저': 6941,
 '지하철역': 7925,
 '마인': 3246,
 '자비스': 7145,
 '중반': 7800,
 '실현': 5546,
 '생태계': 4737,
 '번성': 3993,
 '도태': 2761,
 '시범': 5372,
 '여원': 6048,
 '격화': 1408,
 '조짐': 7657,
 '진출': 7978,
 '세컨드': 4934,
 '메신저': 3413,
 '계좌': 1488,
 '개설': 1301,
 '이모티콘': 6844,
 '곳도': 1617,
 '린지': 3186,
 '전만': 7395,
 '경쟁자': 1456,
 '중립': 7797,
 '선상': 4821,
 '생물': 4719,
 'ceo': 132,
 '주년': 7705,
 '소회': 5027,
 '시너지': 5348,
 '내부': 2267,
 '논의': 2364,
 '액션': 5815,
 '완결': 6354,
 '베스트': 4033,
 '브레인': 4354,
 '노동부': 2320,
 '권리': 1876,
 '다라': 2454,
 '베타서비스': 4043,
 '제지': 7609,
 '하루하루': 9240,
 '실무': 5520,
 '시각': 5329,
 '시행착오': 5421,
 '웹툰': 6591,
 '회화': 9712,
 '감안': 1229,
 '매니저': 3341,
 '역삼동': 6069,
 '데모': 2688,
 '세션': 4923,
 '테크니컬': 8740,
 '전산언어학': 7406,
 '자인': 7164,
 '최현정': 8281,
 '연구원': 6088,
 '킹스': 8634,
 '골든': 1601,
 '서클': 4790,
 '범죄도시': 4009,
 '제품군': 7611,
 '만회': 3298,
 '라인업': 3002,
 '악의': 5701,
 '점수': 7456,
 '필터링': 9213,
 '진미': 7961,
 '열흘': 6162,
 'amp': 25,
 '상거래': 4642,
 '전환율': 7442,
 '격차': 1407,


<br/><br/>
### TF-IDF 적용
- TF * IDF는 특정 문서 내에서 단어 빈도가 높을수록, 전체 문서들에는 그 단어를 포함한 문서가 적을수록 TF * IDF값이 높아지는 특징이 있다.
- 이러한 특징을 이용해서 모든 문서에 나타나는 흔한 단어들을 걸러내며, 특정 단어가 가지는 중요도를 측정하는 데 사용된다.
- 한마디로 TF * IDF 값은 특정 단어가 가지는 중요도!
- TF(Term Frequency): 단어 빈도
   - 해당 문서에서 단어가 나타나는 빈도수
   - 문서의 길이가 길면 해당 단어의 실제 중요도와는 상관없이 단어의 빈도수는 증가될 확률이 높다.
   - 위의 문제를 해결하기 위해 다음과 같이 표준화 -> (문서에서 단어가 나타나는 빈도수) / (모든 단어가 나타나는 빈도수)
- IDF(Inverse Document Frequency): 역문헌 빈도
   - 해당 단어의 일반적인 중요도를 나타내는 값
   - log( 전체 문서의 수 / 해당 단어가 포함된 문서들의 수 )

In [340]:
from sklearn.feature_extraction.text import TfidfTransformer    # Tf * idf 구할 때

In [341]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)        # Tf-idf 가중치를 적용할 수 있도록 변환시켜줌
tf_idf_matrix = tfidf_transformer.fit_transform(word_count_vector)

feature_names=cv.get_feature_names()

dense = tf_idf_matrix.todense()

for i in range(len(dense)):
    doc = dense[i].tolist()[0]           # dense[i].tolist()는 2차원 list, 예를들면 shape: (1,814)
    phrase_scores = [pair for pair in zip(range(0, len(doc)), doc) if pair[1] > 0]      # 0의 값이 아닌것만 모아서 만듦

    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1], reverse=True) # sorted(phrase_scores, key=lambda t: t[1] * -1)라고 해도 됨
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
        print('{0: <20} {1}'.format(phrase, score))      # 단어와 단어의 tf-idf 값을 출력
    print()
    ##### 이 부분은 테스트를 위해서 #####
    if i == 3:     
        break
    ###############################

미니언즈                 0.13642020917091804
펌웨어                  0.13642020917091804
wave                 0.12806463478391397
엘지                   0.098222550603061
반려견                  0.09404059250871988
약관                   0.09276574223622427
보호자                  0.08762317116794112
셀리                   0.07414268329595018
필립스                  0.07093850876887739
int                  0.06839315818815991
엘지유플러스               0.06839315818815991
선풍기                  0.06740243935995471
잠시                   0.06740243935995471
새벽                   0.06548170040204067
키보드                  0.06548170040204067
lg유플러스               0.060662195423959245
가방                   0.060662195423959245
센서                   0.06002489203520395
안함                   0.06002489203520395
외출                   0.06002489203520395

취약점                  0.13934445019108485
핫스팟                  0.10063765847133906
새벽                   0.08515494178344074
펌웨어                  0.07741358343949158
포터블            